In [1]:
import random
import torch
from tqdm import tqdm, trange
import sys
sys.path.append('../')
from model import GNNModel, APPNP
from gen_subg import preprocess_ogb_data
import dgl
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import pdb
import argparse
from torch_geometric.seed import seed_everything
from ogb.nodeproppred import DglNodePropPredDataset
import time
import copy

In [16]:
@torch.no_grad()
def test(model, graph):
    model = model.eval()
    nodes = graph.split_idx['test']
    output = model(graph, graph.ndata['feat'])
    labels = graph.ndata['label'][nodes]
    correct = (output[nodes].argmax(-1) == labels).sum().item()
    total = labels.size(0)
    return correct / total

In [2]:
seed_everything(123)

In [3]:
class object_view:
    def __init__(self, d):
        self.__dict__ = d

In [4]:
args = object_view({'dataset': ''})
input_channels = 100

In [5]:
output_channels = 47
dataset = DglNodePropPredDataset(name=f'ogbn-products', root='/home/ubuntu/dataset')
graph = preprocess_ogb_data(dataset, args)


In [6]:
polluted_idx, _ = torch.load('id-map-0.1.pt')
feat = torch.load('feat_100.pt') 
# graph.ndata['feat'][polluted_idx] = graph.ndata['feat'][mapping]
graph.ndata['feat'][polluted_idx] = feat

In [35]:
graph.ndata['feat'][polluted_idx] *= 2

In [36]:
# model = torch.load('./polluted-gauss.pt') # [4, 0]
model = torch.load('./polluted-gauss100.pt') # [1, 0]
model = model.to(torch.device('cpu'))
model = model.eval()

In [37]:
Y = model(graph, graph.ndata['feat']).detach()
f = model.mlp_bef(graph.ndata['feat']).detach()

In [38]:
diff = torch.norm(Y - f, dim=1)

In [39]:
sorted_diff, indices = torch.sort(diff, descending=True)

In [40]:
#10*N(0,1)  [1,0]
for i in range(1,11):
    print(i,len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100)
# len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100

1 76.45548015124417
2 20.09211848004508
3 0.25357081608153464
4 0.1996717054168606
5 0.1735388032764126
6 0.1743554564683016
7 0.2078382373357506
8 0.2968534352516517
9 0.5545075172926313
10 1.5920653975876065


In [45]:
len(set(indices[244902*0:int(244902*1.4)].tolist()).intersection(set(polluted_idx)))/244902*100

95.63866362871678

In [30]:
#5*N(0,1)  [1,0]
for i in range(1,11):
    print(i,len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100)
# len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100

1 65.32490547239304
2 30.225967938195687
3 1.0069333855991376
4 0.1898718671141926
5 0.1694555373169676
6 0.1947717862655266
7 0.22294632138569714
8 0.3001200480192077
9 0.6165731598761954
10 1.74845448383435


In [34]:
len(set(indices[244902*0:int(244902*1.1)].tolist()).intersection(set(polluted_idx)))/244902*100

72.47102922801774

In [11]:
#100*N(0,1)  [1,0]
for i in range(1,11):
    print(i,len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100)
# len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100

1 80.89113196298928
2 16.64870029644511
3 0.4940751810928453
4 0.35891907783521576
5 0.27725375864631563
6 0.23478779266808766
7 0.1772137426399131
8 0.18456362136691412
9 0.20824656393169513
10 0.5251080023846273


In [25]:
len(set(indices[244902*0:int(244902*1.2)].tolist()).intersection(set(polluted_idx)))/244902*100

96.17969636834326

In [54]:
#5*N(0,1)  [4,0]
for i in range(1,11):
    print(i,len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100)
# len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100

1 62.53848478166777
2 28.69065993744437
3 5.896644372034529
4 1.6990469657250655
5 0.6320895705220864
6 0.2846036373733167
7 0.1359727564495186
8 0.06859886811867605
9 0.034299434059338024
10 0.01959967660533601


In [41]:
#10*N(0,1)  [4,0]
for i in range(1,11):
    print(i,len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100)
# len(set(indices[244902*(i-1):244902*i].tolist()).intersection(set(polluted_idx)))/244902*100

1 76.45548015124417
2 20.09211848004508
3 0.25357081608153464
4 0.1996717054168606
5 0.1735388032764126
6 0.1743554564683016
7 0.2078382373357506
8 0.2968534352516517
9 0.5545075172926313
10 1.5920653975876065
